In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit



<h2>1. What test is appropriate for this problem? Does CLT apply?</h2>

<p>The hypothesis based on categories and percentages usually suggests a $\chi^2$-test, but the $\chi^2$-test requires *frequencies* rather than percentages. </p>

For now, a two-sample $z$-test:

$$z = \frac{\left( \hat{p}_W - \hat{p}_B \right) - 0}{\sqrt{\hat{p} (1 - \hat{p)} \left( \frac{1}{n_W} + \frac{1}{n_B}\right)}}$$

where

$$\hat{p} = \frac{y_W + y_B}{n_W + n_B}$$

CLT applies because we assume that the samples are representative of the population. The observations in each sample are assumed to be independent since the sample was drawn randomly.


<h2>2. What are the null and alternate hypotheses?</h2>

The null and alternate hypotheses:

$$H_0: p_B = p_W$$  
The percentage of callback resumes from assumed black application is the same as that from white. 

$$H_A: p_B \ne p_W$$
The percentage of callback resumes from assumed black application is different from that from white. 


In [5]:
#calculation using frequentist statistical approaches

w = data[data.race=='w']
b = data[data.race=='b']

n_w = len(w)
n_b = len(b)

prop_w = np.sum(w.call) / len(w)
prop_b = np.sum(b.call) / len(b)

prop_diff = prop_w - prop_b
phat = (np.sum(w.call) + np.sum(b.call)) / (len(w) + len(b))

z = prop_diff / np.sqrt(phat * (1 - phat) * ((1 / n_w) + (1 / n_b)))
pval = stats.norm.cdf(-z) * 2
print("Z score: {}".format(z))
print("P-value: {}".format(pval))

moe = 1.96 * np.sqrt(phat * (1 - phat) * ((1 / n_w) + (1 / n_b)))
ci = prop_diff + np.array([-1, 1]) * moe
print("Margin of Error: {}".format(moe))
print("Confidence interval: {}".format(ci))

Z score: 4.108412152434346
P-value: 3.983886837585077e-05
Margin of Error: 0.015281912310894095
Confidence interval: [ 0.01675094  0.04731477]


In [6]:
#calculation using bootstraping statistical approaches

wcall = w['call']
bcall = b['call']

def frac_data(data1):
    """Compute fraction of white callbacks."""
    frac_1 = np.sum(data1) / len(data1)
    return frac_1

def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1,data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[0:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)
    bs_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1,data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1)
        
        # Compute the test statistic 2
        bs_replicates[i] = func(np.random.choice(data_1, size=len(data_1)))-func(np.random.choice(data_2, size=len(data_2)))
        

    return perm_replicates, bs_replicates

# Acquire permutation samples: perm_replicates
perm_replicates, bs_replicates = draw_perm_reps(wcall, bcall, frac_data, 100000)

ci = np.percentile(bs_replicates,[0.5,99.5])
print("Confidence interval: ", ci)

# Compute and print p-value: p
p = np.sum(perm_replicates > frac_data(wcall)) / len(perm_replicates)
print('p-value =', p)


Confidence interval:  [ 0.01190965  0.05215606]
p-value = 0.0



<h2>3. Compute margin of error, confidence interval, and p-value. </h2>

<p>Calculation using the bootstrapping and frequentist statistcal approaches gives the same result: the p-value is very small, practically 0. so we reject the null hypothesis that white and black sounding names have the same callback rate. They do not.<p>
<p>Since 0 is not in the confidence interval, we reject the null hypothesis with the same conclusion.</p>




<h2>4. Write a story describing the statistical significance in the context or the original problem. </h2>
#### A study of racial disparity in callback rate in US job market
<p>The null hypothesis is defined as $p_w=p_b$, the callback rate for the resume is the same for black-sounding and white sounding names.<p>
<p>We have calculated the p-value using the bootstrapping and frequenist statistical methods. Both gave practically 0 p-value. We have chosen significance level $\alpha = 0.01$. From $p < \alpha $, the null hypothesis is rejected. <p>
<p>It could also be noticed from the confidence interval that the difference of the callback rate from white_sounding names and black_sounding names does not include 0. This concludes the same theory, there is a difference in how employers behave when treating a black or white sounding names. <p> 
<p>The conclusion: the callback rate is different for black-sounding and white-sounding names in the US job market. 




<h2>Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?</h2>

<p>No. While the test did show that there is a difference in callback rate based on race alone, there are other variables that may also contribute to, or interact with, race to explain the difference. In the original research paper, the researchers cited geography/city as a confounding variable. Additionally, the education and experience levels could be taken into considerations as well. But, in this very narrow example, we have shown that there is a significant difference in callback rates between white people and black people.</p>

<p>The $\chi^2$ approach:</p>
</div>

In [7]:
cont_table = pd.crosstab(index=data.call, columns=data.race)
chi2, pval, _, _ = stats.chi2_contingency(cont_table)
print("Chi-squared test statistic: {}".format(chi2))
print("p-value: {}".format(pval))

Chi-squared test statistic: 16.44902858418937
p-value: 4.997578389963255e-05



The chi-squared test yields a similar result. We reject the null hypothesis that race and callback rate are independent. The margin of error and confidence interval calculations are a bit more complicated because the chi-squared distribution is not always symmetric, depending on the number of degrees of freedom.
</div>